In [8]:
#Defining function to plot centroid distances over the course of the simulation

import matplotlib.pyplot as plt
import pandas as pd
import os
import math

# Directory containing the .xyz files to analyze 
directory = '/home/mfi/Desktop/mfi/MIL-68Ga-guest'

# List of dataframes
dfs = []

# Save lattice parameters in seperate dataframe
latticefilename = 'MIL68Ga-guest-02.xyz'
lattice = pd.read_csv(os.path.join(directory,latticefilename), delimiter='\s+', header=None, skiprows=range(2,400000), nrows=1)
numAtoms = lattice[0][0]-1
print(numAtoms)

# Save indices of carbons of which to form centroids
indexfilename = "indicessorted.dat"
centroid_indices = pd.read_csv(os.path.join(directory, indexfilename), delimiter = " ", header = None)
#print(centroid_indices)
numOfCentroides  = centroid_indices.index.size
#print(numOfCentroides)


720


In [9]:
#Loop through xyz files
for filename in sorted(os.listdir(directory)):
    if filename.endswith('.xyz') and filename.startswith('MIL68Ga-guest'):
        #print(filename)
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, delimiter='\s+', header=None, skiprows=1)
        df['run'] = filename
        #print(df[0][1])
        # df['cleanedIndex'] = (df.index)%(numAtoms)
        #drop all rows not containing coordinates
        df = df[~df.iloc[:,1].isna()]
        #drop all rows that are an X element
        df = df[df[0] != 'X']
        df.reset_index(drop=True, inplace=True)
        #add cleanedIndex to each row representing its index in the simulation, going from 1 to numAtoms. Number rows from 1 to numAtoms, then start at 1 again
        df['cleanedIndex'] = (df.index)%(numAtoms)+1
        #df['runningIndex'] = ((df.index)//(numAtoms+1)+1).astype(int)
        print(df)
        dfs.append(df)
        

#Concat dataframes into one big dataframe, containing all the coordinates of all the simulations. Keep cleanedIndex to identify atoms
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

          0        1        2       3                   run  cleanedIndex
0        Ga  10.8656  -0.0717 -3.5586  MIL68Ga-guest-01.xyz             1
1        Ga  10.2688  -0.1293 -7.0245  MIL68Ga-guest-01.xyz             2
2        Ga  -0.3134  18.6673 -3.2842  MIL68Ga-guest-01.xyz             3
3        Ga   0.2141 -18.5028 -6.6460  MIL68Ga-guest-01.xyz             4
4        Ga  -4.4561   9.2758 -3.7500  MIL68Ga-guest-01.xyz             5
...      ..      ...      ...     ...                   ...           ...
1799995   H  -0.6494  -6.0928  3.9265  MIL68Ga-guest-01.xyz           716
1799996   C  -2.5494  -5.5943  4.7929  MIL68Ga-guest-01.xyz           717
1799997   H  -1.9637  -4.8087  5.2712  MIL68Ga-guest-01.xyz           718
1799998   H  -3.5062  -5.1469  4.5226  MIL68Ga-guest-01.xyz           719
1799999   H  -2.6228  -6.4228  5.4978  MIL68Ga-guest-01.xyz           720

[1800000 rows x 6 columns]
          0        1        2       3                   run  cleanedIndex
0        G

In [10]:
#Add new index assigning each row to its respective step in the simulation. First 720 should be 1, next 720 rows should be 2, etc.
merged_df['runningIndex'] = ((merged_df.index)//(numAtoms)).astype(int)+1

In [11]:
#Find max value of runningIndex column, i.e. number of steps performed in total
numSteps = merged_df['runningIndex'].max()
print(numSteps)

43750


In [33]:
# Loop over the simulation and calculate the coordinates of each centroid (not weighted). i is the step number given by the runningIndex column, j is the centroid number that is looped over in each step
for i in range(1, 5):
    stepframe = merged_df[merged_df['runningIndex'] == i]
    for j in range(1, numOfCentroides + 1):
        centroid_indices_j = centroid_indices.loc[j - 1, :]
        stepframe_j = stepframe[stepframe['cleanedIndex'].isin(centroid_indices_j)]
        merged_df.loc[merged_df['runningIndex'] == i, 'x_coords_of_centroid ' + str(j)] = (stepframe_j[1] ** 2).sum() ** 0.5
        merged_df.loc[merged_df['runningIndex'] == i, 'y_coords_of_centroid ' + str(j)] = (stepframe_j[2] ** 2).sum() ** 0.5
        merged_df.loc[merged_df['runningIndex'] == i, 'z_coords_of_centroid ' + str(j)] = (stepframe_j[3] ** 2).sum() ** 0.5

print(merged_df.loc[merged_df['runningIndex'] == 1])

      0        1        2       3                   run  cleanedIndex  \
0    Ga  10.8656  -0.0717 -3.5586  MIL68Ga-guest-01.xyz             1   
1    Ga  10.2688  -0.1293 -7.0245  MIL68Ga-guest-01.xyz             2   
2    Ga  -0.3134  18.6673 -3.2842  MIL68Ga-guest-01.xyz             3   
3    Ga   0.2141 -18.5028 -6.6460  MIL68Ga-guest-01.xyz             4   
4    Ga  -4.4561   9.2758 -3.7500  MIL68Ga-guest-01.xyz             5   
..   ..      ...      ...     ...                   ...           ...   
715   H  -2.3503  -0.6173  0.0901  MIL68Ga-guest-01.xyz           716   
716   C  -0.6275  -0.7401  1.2288  MIL68Ga-guest-01.xyz           717   
717   H  -0.0949  -0.1026  0.5226  MIL68Ga-guest-01.xyz           718   
718   H  -0.9725  -0.0521  2.0011  MIL68Ga-guest-01.xyz           719   
719   H   0.1590  -1.3488  1.6757  MIL68Ga-guest-01.xyz           720   

     runningIndex  x_coords_of_centroid 1  y_coords_of_centroid 1  \
0               1               19.524544             

In [35]:
print(merged_df[merged_df['runningIndex'] == 3])

       0        1        2       3                   run  cleanedIndex  \
1440  Ga  10.8744  -0.0660 -3.5796  MIL68Ga-guest-01.xyz             1   
1441  Ga  10.2553  -0.1290 -7.0295  MIL68Ga-guest-01.xyz             2   
1442  Ga  -0.3249  18.6843 -3.2864  MIL68Ga-guest-01.xyz             3   
1443  Ga   0.2282 -18.5060 -6.6500  MIL68Ga-guest-01.xyz             4   
1444  Ga  -4.4368   9.2826 -3.7398  MIL68Ga-guest-01.xyz             5   
...   ..      ...      ...     ...                   ...           ...   
2155   H  -2.4644  -0.8680 -0.0192  MIL68Ga-guest-01.xyz           716   
2156   C  -0.6033  -0.7397  1.2390  MIL68Ga-guest-01.xyz           717   
2157   H  -0.1038  -0.1221  0.4921  MIL68Ga-guest-01.xyz           718   
2158   H  -1.0471  -0.1293  2.0259  MIL68Ga-guest-01.xyz           719   
2159   H   0.0486  -1.4704  1.7184  MIL68Ga-guest-01.xyz           720   

      runningIndex  x_coords_of_centroid 1  y_coords_of_centroid 1  \
1440             3               19.53296

In [17]:
#Define centroid of the guest molecule by the indices of it's carbon ring
guest_indices = [686, 687, 688, 689, 690, 685]

In [20]:
# Loop over every step of the simulation
for i in range(1, 10000, 100):
    # Get the stepframe for the current step
    stepframe = merged_df[merged_df['runningIndex'] == i]
        
    # Calculate the Guest coordinates for the current step
    x_coords = ((stepframe.loc[stepframe['cleanedIndex'].isin(guest_indices), 1]) ** 2).sum() ** 0.5
    y_coords = ((stepframe.loc[stepframe['cleanedIndex'].isin(guest_indices), 2]) ** 2).sum() ** 0.5
    z_coords = ((stepframe.loc[stepframe['cleanedIndex'].isin(guest_indices), 3]) ** 2).sum() ** 0.5
        
    # Write the Guest coordinates into new columns
    merged_df.loc[merged_df['runningIndex'] == i, 'Guest_x_coords'] = x_coords
    merged_df.loc[merged_df['runningIndex'] == i, 'Guest_y_coords'] = y_coords
    merged_df.loc[merged_df['runningIndex'] == i, 'Guest_z_coords'] = z_coords


In [33]:
print(merged_df.loc[merged_df['runningIndex'] == 1])

      0        1        2       3                   run  cleanedIndex  \
0    Ga  10.8656  -0.0717 -3.5586  MIL68Ga-guest-01.xyz             1   
1    Ga  10.2688  -0.1293 -7.0245  MIL68Ga-guest-01.xyz             2   
2    Ga  -0.3134  18.6673 -3.2842  MIL68Ga-guest-01.xyz             3   
3    Ga   0.2141 -18.5028 -6.6460  MIL68Ga-guest-01.xyz             4   
4    Ga  -4.4561   9.2758 -3.7500  MIL68Ga-guest-01.xyz             5   
..   ..      ...      ...     ...                   ...           ...   
715   H  -2.3503  -0.6173  0.0901  MIL68Ga-guest-01.xyz           716   
716   C  -0.6275  -0.7401  1.2288  MIL68Ga-guest-01.xyz           717   
717   H  -0.0949  -0.1026  0.5226  MIL68Ga-guest-01.xyz           718   
718   H  -0.9725  -0.0521  2.0011  MIL68Ga-guest-01.xyz           719   
719   H   0.1590  -1.3488  1.6757  MIL68Ga-guest-01.xyz           720   

     runningIndex  x_coords_of centroid 1  y_coords_of centroid 1  \
0               1               19.524544             

In [37]:
def calculateDistanceToCentroid(stepframe, centroidNumber):
    x_distance = stepframe['Guest_x_coords'] - stepframe['x_coords_of centroid ' + str(centroidNumber)]
    y_distance = stepframe['Guest_y_coords'] - stepframe['y_coords_of centroid ' + str(centroidNumber)]
    z_distance = stepframe['Guest_z_coords'] - stepframe['z_coords_of centroid ' + str(centroidNumber)]
    distance = ((x_distance**2 + y_distance**2 + z_distance**2)**0.5).astype(float)
    return distance

In [34]:
# Finally, in each step, calculate the distance between each centroid and the guest molecule
for i in range(1, 2):
    # Get the stepframe for the current step
    stepframe = merged_df[merged_df['runningIndex'] == i]
    for j in range(1, numOfCentroides + 1):
        x_distance = stepframe['Guest_x_coords'] - stepframe['x_coords_of_centroid ' + str(j)]
        y_distance = stepframe['Guest_y_coords'] - stepframe['y_coords_of_centroid ' + str(j)]
        z_distance = stepframe['Guest_z_coords'] - stepframe['z_coords_of_centroid ' + str(j)]
        distance = ((x_distance ** 2 + y_distance ** 2 + z_distance ** 2) ** 0.5).astype(float)
        merged_df.loc[merged_df['runningIndex'] == i, 'distance_to_centroid ' + str(j)] = distance
    
        
   

In [ ]:
import matplotlib.pyplot as plt

# Filter the dataframe to include only rows with non-null distance values
filtered_df = merged_df.dropna(subset=[f'distance_to_centroid {j}' for j in range(1, numOfCentroides + 1)])

# Plot the distance against the step number for each centroid
for j in range(1, numOfCentroides + 1):
    plt.scatter(filtered_df['runningIndex'], filtered_df[f'distance_to_centroid {j}'], label=f'Centroid {j}')

plt.xlabel('Step Number')
plt.ylabel('Distance')
plt.legend()
plt.show()




In [39]:
print(merged_df.loc[merged_df['runningIndex'] == 1])

      0        1        2       3                   run  cleanedIndex  \
0    Ga  10.8656  -0.0717 -3.5586  MIL68Ga-guest-01.xyz             1   
1    Ga  10.2688  -0.1293 -7.0245  MIL68Ga-guest-01.xyz             2   
2    Ga  -0.3134  18.6673 -3.2842  MIL68Ga-guest-01.xyz             3   
3    Ga   0.2141 -18.5028 -6.6460  MIL68Ga-guest-01.xyz             4   
4    Ga  -4.4561   9.2758 -3.7500  MIL68Ga-guest-01.xyz             5   
..   ..      ...      ...     ...                   ...           ...   
715   H  -2.3503  -0.6173  0.0901  MIL68Ga-guest-01.xyz           716   
716   C  -0.6275  -0.7401  1.2288  MIL68Ga-guest-01.xyz           717   
717   H  -0.0949  -0.1026  0.5226  MIL68Ga-guest-01.xyz           718   
718   H  -0.9725  -0.0521  2.0011  MIL68Ga-guest-01.xyz           719   
719   H   0.1590  -1.3488  1.6757  MIL68Ga-guest-01.xyz           720   

     runningIndex  x_coords_of centroid 1  y_coords_of centroid 1  \
0               1               19.524544             

In [ ]:

def plot_file_data(file_path, x_column, y_column,Title, xtitle,ytitle):
    x = []
    y = []

    with open(file_path, 'r') as file:
        for line in file:
            data = line.split()
            x.append(float(data[x_column]))
            y.append(float(data[y_column]))

    if len(x) == 0 or len(y) == 0:
        print("One or more columns are empty.")
        return
    
    plt.scatter(x,y,martker='x')
    plt.title(Title)
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)
    plt.show